## 윈도우 함수 (WINDOW 함수)
- 행과 행간의 관계를 정의하거나, 행과 행간을 비교 연산하는 등의 SQL문을 작성할 때 사용하는 WINDOW 함수
- 분석함수나 순위함수로도 잘 알려져 있는 윈도우 함수는 DW에서 발전한 기능이다.

1. 그룹 내 순위(RANK) : RANK, DENSE_RANK, ROW_NUMBER
2. 그룹 내 집계(AGGREGATION) : SUM, MAX, MIN, AVG, COUNT
3. 그룹 내 행 순서 : FIRST_VALUE, LAST_VALUE, LAG, LEAD
4. 그룹 내 비율 : CUME_DIST, PERCENT_RANK, NTILE, RATIO_TO_REPORT
5. 선형 분석을 포함한 통계 분석 관련 함수 (통계와 관련된 특화 기능이므로 본 가이드에서는 생략되었다.)

### WINDOW FUNCTION SYNTAX
- WINDOW 함수에는 OVER 문구가 키워드로 필수 포함된다.

In [ ]:
SELECT WINDOW_FUNCTION (ARGUMENTS) OVER
(|PARTITION BY 칼럼| |ORDER BY 칼럼| |WINDOWING 절|)
FROM 테이블 명;

- WINDOW_FUNCTION    
기존에 사용하던 함수도있고, 새롭게 WINDOW용으로 추가된 함수도 있다.   


- ARGUMENTS(인수)   
함수에 따라 0~N개의 인수가 지정될 수 있다.


- PARTITION BY 절   
전체 집합을 기준에 의해 소그룹으로 나눌 수 있다.


- ORDER BY 절   
어떤 항목에 대해 순위를 지정할 지 ORDER BY 절을 기술한다.


- WINDOWING 절   
WINDOWING 절은 함수의 대상이 되는 행 기준의 범위를 강력하게 지정할 수 있다. ROWS는 물리적인 결과 행의 수를, RANGE는 논리적인 값에 의한 범위를 나타내는데, 둘 중의 하나를 선택해서 사용할 수 있다.

## 1. 그룹 내 순위 함수
### 1-1 RANK 함수
RANK 함수는 ORDER BY를 포함한 QUERY 문에서 특정 항목(칼럼)에 대한 순위를 구하는 함수이다.

이 떄 특정 범위(PARTITION) 내에서 순위를 구할 수도 있고 전체 데이터에 대한 순위를 구할 수도 있다.(동일한 값에 대해서는 동일한 순위를 부여)

In [ ]:
SELECT JOB, ENAME, SAL,
    RANK() OVER (ORDER BY SAL DESC) ALL_RANK,
    RANK() OVER (PARTITION BY JOB ORDER BY SAL DESC) JOB_RANK
FROM EMP;

하나의 SQL 문장에 ORDER BY SAL DESC 조건과 PARTITION BY JOB 조건 충돌

    → JOB별로는 정렬이 되지 않고, ORDER BY SAL DESC 조건으로 정렬됨.
    
- 1번 RANK : SAL 기준으로 정렬한 뒤 전체 행에 대한 순위를 ALL_RANK 칼럼으로 형성
- 2번 RANK : SAL 기준으로 정렬 된 전체 행을 같은 JOB 범위 내에서만 순위를 구해 JOB_RANK 칼럼으로 형성

In [ ]:
SELECT JOB, ENAME, SAL,
    RANK() OVER (PARTITION BY JOB ORDER BY SAL DESC) JOB_RANK
FROM EMP;

업무별로 SAL 순서를 구하는 JOB_RANK만 사용한 경우 파티션의 기준이 된 JOB과 SALARY별로 정렬된다.

JOB의 묶음은 출력되면서 가로줄이 도식화되어 시각적으로 구분됨

### 1-2 DENSE_RANK 함수
동일한 순위를 하나의 건수로 취급하는 특징이 있는 함수이다.

In [ ]:
SELECT JOB, ENAME, SAL,
    RANK() OVER (ORDER BY SAL DESC) RANK,
    DENSE_RANK() OVER (ORDER BY SAL DESC) DENSE_RANK
FROM EMP;

RANK 함수 사용시 : 1위, 2위(공동), 2위(공동), **4위**, ...

DENSE_RANK 사용시: 1위, 2위(공동), 2위(공동), **3위**, ...

### 1-3 ROW_NUMBER 함수
동일한 값이라고 하더라도 고유한 순위를 부여하는 함수이다.

In [ ]:
SELECT JOB, ENAME, SAL,
    RANK() OVER (ORDER BY SAL DESC) RANK,
    ROW_NUMBER() OVER (ORDER BY SAL DESC) ROW_NUMBER
FROM EMP;

RANK 함수 사용시 : 1위, 2위(공동2위), **2위(공동2위)**, 4위, ...

ROW_NUMBER 사용시: 1위, 2위(공동2위), **3위(공동2위)**, 3위, ...

＊데이터베이스별로 틀린 결과가 나올 수 있다. (공통된 값을 가질 경우 어떤 ROW가 먼저 출력되어 더 높은 순위가 나올지 모름)    

    → ORDER BY 절을 이용해 추가적인 정렬 기준을 정의해야 한다. 예) ROW_NUMBER() OVER (ORDER BY SAL DESC, EMP) ROW_NUMBER
    
    → ORACLE의 경우 ROWID가 적은 행이 먼저 나온다.

---
## 2. 일반 집계 함수
### 2-1 SUM함수
파티션별 윈도우의 합을 구할 수 있다.

In [ ]:
SELECT MGR, ENAME, SAL,
    SUM(SAL) OVER (PARTITION BY MGR) MGR_SUM
FROM EMP;

PARTITION 으로 설정된 MGR을 기준으로 파티션화 된 후 같은 그룹으로 묶인 행의 'SAL' 값을 모두 합해 MGR_SUM 칼럼에 출력한다.
- 모든 행의 MGR_SUM 칼럼에 출력된다. (별도로 SUM 결과값 출력행이 형성되거나 가장 마지막 행에만 출력되는 등 하지 않음)

OVER 절 내에 ORDER BY 절을 추가해 파티션 내 데이터를 정렬하고 이전 SQL까지의 누적값을 출력하기

RANGE UNBOUNDED PRECEDING → 현재 행을 기준으로 파티션 내의 첫 번째 행까지의 범위를 지정한다.

In [ ]:
SELECT MGR, ENAME, SAL,
    SUM(SAL) OVER (PARTITION BY MGR ORDER BY SAL RANGE UNBOUNDED PRECEDING)as MGR_SUM
FROM EMP;

### 2-2 MAX 함수
파티션별 윈도우의 최댓값을 구할 수 있다.

In [ ]:
SELECT MGR, ENAME, SAL, 
    MAX(SAL) OVER (PARTITION BY MGR) as MGR_MAX
FROM EMP;
# MGR 별로 PARTITION 했기 때문에, MGR에 따른 사원들의 SALARY 중 최댓값을 출력한다.

In [ ]:
# INLINE VIEW를 활용해 파티션별 최댓값을 가진 행만 추출하기

SELECT MGR, ENAME, SAL 
FROM (SELECT MGR, ENAME, SAL, MAX(SAL) OVER (PARTITION BY MGR) as IV_MAX_SAL
      FROM EMP)
WHERE SAL = IV_MAX_SAL;

# 공동으로 높은 값을 가질 경우 해당하는 행이 모두 함께 출력된다.

### 2-3 MIN 함수
파티션별 윈도우의 최소값을 구할 수 있다.

In [ ]:
SELECT MGR, ENAME, SAL, 
    MIN(SAL) OVER (PARTITION BY MGR ORDER BY HIREDATE) as MGR_MIN
FROM EMP;

#같은 MGR별 채용 일자가 빠른 순으로 정렬되며, 그 중에서 SALARY가 가장 낮은 값을 MGR_MIN 칼럼에 값으로 출력한다.

### 2-4 AVG 함수
AVG 함수와 파티션별 ROWS 윈도우를 이용해 원하는 조건에 맞는 데이터에 대한 통계값을 구할 수 있다.

In [ ]:
# 같은 매니저 내에서 자기 바로 앞의 사번과 바로 뒤의 사번인 직원만을 대상으로 평균 SALARY 구하기

SELECT MGR, ENAME, HIREDATE, SAL,
    ROUND (AVG(SAL) OVER (PARTITION BY MGR ORDER BY HIREDATE
                         ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING)) as MGR_AVG
FROM EMP;

#만약 PARTITION 내에서 자기 앞 혹은 뒤의 데이터가 없는 경우, 본인과 조건에 맞는 데이터만 사용하여 평균을 계산한다.

### 2-5 COUNT 함수
COUNT함수와 파티션별 ROWS 윈도우를 이용해 원하는 조건에 맞는 데이터에 대한 통계값을 구할 수 있다.

In [ ]:
SELECT ENAME, SAL, 
    COUNT(*) OVER (ORDER BY SAL RANGE BETWEEN 50 PRECEDING AND 150 FOLLOWING) as SIM_CNT
FROM EMP;

# RANGE BETWEEN 50 PRECEDING AND 150 FOLLOWING : 현재 행의 급여값을 기준으로 급여가 -50에서 +150 범위 내에 포함된 모든 행이 대상이 된다.
# (현재 행의 값을 기준으로 앞뒤 데이터 값의 범위를 표시하는것!)